# "TensorFlow for Mac OS Quick Start"
> "This notebook shows a very quick example of how to run standard TensorFow 2.0 code in the Tensorflow for MacOS environment. The code in this notebook is basically a port of the TensorFlow 2 quickstart for beginners from `https://www.tensorflow.org/tutorials/quickstart/beginner`. The code is mostly unchaged from the original tutorial. I have added in a couple of places comments and/or print statements to check and see what is going on. The goal is to (hopefully) demonstrate that you should be able (for the most part) take regular Tensorflow 2.0 code and run it under the Mac OS implementation."

- toc: true
- branch: master
- badges: true
- comments: true
- author: tensorflow.org, Ed Martinez
- categories: [tensorflow, MacOS, keras]


## References

* Tensorflow 2 Quickstart. https://www.tensorflow.org/tutorials/quickstart/beginner

## Setup environment

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.python.compiler.mlcompute import mlcompute
mlcompute.set_mlc_device(device_name = 'gpu')

In [3]:
print(tf.__version__)

2.4.0-rc0


## Run the code

This example will use a dataset that is part of TF 2.0 MNIST. Load the dataset and convert the samples from integers to floating point numbers. 

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

The example uses a very simple model - The Keras `sequential` model. Build the tf.keras.Sequential model by stacking layers. Choose an optimizer and loss function for training:

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.9982459 ,  0.35465944,  0.0547075 , -0.06532808, -0.09845451,
        -0.8171675 , -0.52887785,  0.00635976, -0.0428137 ,  0.31171066]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class:

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.04070023, 0.1574549 , 0.11665107, 0.10345653, 0.10008553,
        0.04877959, 0.06507899, 0.11114541, 0.1058122 , 0.15083557]],
      dtype=float32)


The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a True index and returns a scalar loss for each example.

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.log(1/10) ~= 2.3.

In [9]:
loss_fn(y_train[:1], predictions).numpy()

3.0204432

Compile the model

In [10]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Now fit the model

In [11]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 47s 24ms/step - loss: 0.4854 - accuracy: 0.8572
Epoch 2/5
1875/1875 [==============================] - 48s 26ms/step - loss: 0.1519 - accuracy: 0.9550
Epoch 3/5
1875/1875 [==============================] - 50s 27ms/step - loss: 0.1104 - accuracy: 0.9662
Epoch 4/5
1875/1875 [==============================] - 47s 25ms/step - loss: 0.0900 - accuracy: 0.9717
Epoch 5/5
1875/1875 [==============================] - 48s 26ms/step - loss: 0.0742 - accuracy: 0.9773


The `Model.evaluate` method checks the models performance, usually on a "Validation-set" or "Test-set".

In [12]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 5s - loss: 0.0693 - accuracy: 0.9781


[0.0692838728427887, 0.9781000018119812]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [13]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [14]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[5.12671534e-07, 3.93867783e-09, 9.53463939e-07, 2.96221348e-04,
        1.17695464e-10, 1.58727730e-07, 1.33805597e-11, 9.99693453e-01,
        5.25140251e-07, 8.21285812e-06],
       [5.30549357e-07, 1.09513676e-04, 9.99669909e-01, 2.14899832e-04,
        7.75622188e-12, 3.36039221e-07, 6.95063633e-08, 3.82515859e-11,
        4.87999478e-06, 3.69983266e-10],
       [4.21975045e-07, 9.99065220e-01, 1.53246379e-04, 1.77207312e-05,
        1.77628808e-05, 3.08062681e-06, 7.81090694e-06, 6.43892388e-04,
        9.00170926e-05, 7.40005873e-07],
       [9.99787033e-01, 1.03599439e-06, 1.09403569e-04, 8.73194494e-07,
        5.92203037e-07, 2.30637725e-05, 4.90020193e-06, 3.01722612e-05,
        4.84664220e-08, 4.29155552e-05],
       [1.21631429e-05, 6.49704290e-09, 4.19733296e-05, 2.69434622e-07,
        9.94199276e-01, 1.50252458e-06, 6.87295824e-06, 6.19247730e-05,
        8.52300127e-06, 5.66747272e-03]], dtype=float32)>